In [ ]:
!sudo apt -y update
!sudo apt install -y wget curl unzip
!wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
!dpkg -i libu2f-udev_1.1.4-1_all.deb
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
!wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
!unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver
!mv /tmp/chromedriver /usr/bin/chromedriver
!pip install selenium
!apt-get update -qq
!apt-get install -y fonts-nanum*
!apt-get install fonts-nanum* -qq

In [ ]:
!pip install konlpy

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup
import re

import sys
if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import pandas as pd
import time

In [ ]:
import pickle

with open('df_kyob.pkl', 'rb') as f: # 교보문고 파일 가져오기
    kyob = pickle.load(f)
print(kyob)

In [ ]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--headless") # GUI 화면을 띄울지 유무. 보통 코딩 완료후에는 필요없기 때문에 끈다
options.add_argument('--disable-dev-shm-usage') # 크롬 메모리 제한 푸는것
options.add_argument("--no-sandbox")
user_agent = "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"
options.add_argument('user-agent=' + user_agent)

driver = webdriver.Chrome(options=options)

In [ ]:
from tqdm import tqdm

book_info = []

for n in tqdm(kyob['판매상품ID'][746:]) :
    url = f'https://product.kyobobook.co.kr/detail/{n}' # 책 페이지
    driver.get(url)
    time.sleep(3)

    try :
        genre = driver.find_element(By.CLASS_NAME, 'category_list_item').text # 책 장르
        contents = driver.find_element(By.CLASS_NAME, 'intro_bottom').text # 책 소개
        index = driver.find_element(By.CLASS_NAME, 'book_contents_item').text # 책 목차
        book_info.append([n, genre, contents, index])

    except :
        book_info.append([n])
        continue

In [ ]:
book = pd.DataFrame(book_info)

In [ ]:
book.columns = ['판매상품ID', '분야', '책 소개', '책 목차']

In [ ]:
book = book.reset_index()
del book['index'] # 리셋하면서 새로생긴 index열 삭제
book

In [ ]:
book_final = pd.concat([kyob, book], axis= 1)

In [ ]:
del book_final[13] # 중복되는 판매상품ID열 삭제
book_final

In [ ]:
book_final.columns = ['순위', '판매상품ID', '상품코드', '상품명', '저자', '출판사', '분야', '발행(출시)일자', '정가',
                      '판매가', '할인율', '적립율', '적립예정포인트', '분야', '책 소개', '책 목차']

In [ ]:
book_final.to_pickle('book_data_final.pkl')